# Challenge 2 | RAG

In [13]:
import subprocess
import time

# --- CONFIGURATION ---
PROJECT_ID = "qwiklabs-gcp-03-ba43f2730b93"
# The exact email from your error log
SERVICE_ACCOUNT = "bqcx-281600971548-ntww@gcp-sa-bigquery-condel.iam.gserviceaccount.com"

print(f"--- Force-Granting Vertex AI User Role to: {SERVICE_ACCOUNT} ---")

# Construct the gcloud command
# We use ! (bang) syntax to run it directly in the shell for reliability
command = f"gcloud projects add-iam-policy-binding {PROJECT_ID} --member='serviceAccount:{SERVICE_ACCOUNT}' --role='roles/aiplatform.user'"

# Run it using the shell magic
get_ipython().system(command)

print("\n⏳ Waiting 60 seconds for permissions to propagate...")
print("Do not skip this wait. IAM needs time to sync across Google's global servers.")
time.sleep(60)
print("✅ Ready to try again.")

--- Force-Granting Vertex AI User Role to: bqcx-281600971548-ntww@gcp-sa-bigquery-condel.iam.gserviceaccount.com ---
Updated IAM policy for project [qwiklabs-gcp-03-ba43f2730b93].
bindings:
- members:
  - serviceAccount:service-281600971548@gcp-sa-vertex-nb.iam.gserviceaccount.com
  role: roles/aiplatform.colabServiceAgent
- members:
  - serviceAccount:service-281600971548@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-281600971548@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-281600971548-ntww@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:qwiklabs-gcp-03-ba43f2730b93@qwiklabs-gcp-03-ba43f2730b93.iam.gserviceaccount.com
  role: roles/bigquery.admin
- members:
  - serviceAccount:281600971548@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - se

In [ ]:
# dataframe:
# uuid: E11F9AD6-D645-4C79-BBED-BA8FC8234625
# output_variable:
# config_str:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='', uuid='E11F9AD6-D645-4C79-BBED-BA8FC8234625')

In [16]:
import vertexai
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel
import time

# --- CONFIGURATION ---
PROJECT_ID = "qwiklabs-gcp-03-ba43f2730b93"
REGION = "us-central1"
DATASET_ID = "aurora_bay_rag"
CONNECTION_ID = "us-central1.vertex-ai-conn"
CSV_URI = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

client = bigquery.Client(project=PROJECT_ID, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

def run_query(sql):
    try:
        job = client.query(sql, location=REGION)
        result = job.result()
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("🚀 Starting Challenge 2 Solution (Final Fix)...")

# --- STEP 1: LOAD DATA WITH EXPLICIT SCHEMA ---
print(f"--- 1. Loading Data into {DATASET_ID} ---")
dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset.location = REGION
client.create_dataset(dataset, exists_ok=True)

table_ref = client.dataset(DATASET_ID).table("faqs_raw")

# FIX: Define the schema manually to guarantee column names
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("question", "STRING"),
        bigquery.SchemaField("answer", "STRING"),
    ],
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1, # Skip the header row in the file
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

load_job = client.load_table_from_uri(CSV_URI, table_ref, job_config=job_config)
load_job.result()
print(f"✅ Loaded {load_job.output_rows} rows from GCS.")

# --- STEP 2: CREATE EMBEDDING MODEL ---
print(f"--- 2. Creating Remote Model (Connection: {CONNECTION_ID}) ---")
create_model_sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`
REMOTE WITH CONNECTION `{PROJECT_ID}.{CONNECTION_ID}`
OPTIONS (ENDPOINT = 'text-embedding-004');
"""
run_query(create_model_sql)

# Give it 5 seconds to propagate
time.sleep(5)

# --- STEP 3: GENERATE EMBEDDINGS ---
print("--- 3. Generating Vectors (Index) ---")
# Now that we forced the schema, 'question' and 'answer' are guaranteed to exist
index_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.faq_vectors` AS
SELECT
  base.question,
  base.answer,
  emb.ml_generate_embedding_result as embedding
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
  (
    SELECT
      question,
      answer,
      CONCAT('Question: ', question, ' Answer: ', answer) as content
    FROM `{PROJECT_ID}.{DATASET_ID}.faqs_raw`
  )
) as emb
JOIN `{PROJECT_ID}.{DATASET_ID}.faqs_raw` as base
ON emb.question = base.question;
"""
run_query(index_sql)
print("✅ Embeddings generated successfully.")

# --- STEP 4: RAG CHATBOT ---
def ask_aurora_bay(user_query):
    print(f"\nUser Query: '{user_query}'")

    # A. Search
    search_sql = f"""
    SELECT
        base.answer,
        (1 - distance) AS similarity_score
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET_ID}.faq_vectors`,
        'embedding',
        (
            SELECT ml_generate_embedding_result, '{user_query}' AS query
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
                (SELECT '{user_query}' AS content)
            )
        ),
        top_k => 1
    )
    ORDER BY similarity_score DESC;
    """
    results = run_query(search_sql)

    # B. Generate
    if results:
        for row in results:
            if row.similarity_score > 0.4:
                prompt = f"""
                You are the Aurora Bay Town Clerk. Answer using this official info ONLY.
                INFO: {row.answer}
                QUESTION: {user_query}
                """
                model = GenerativeModel("gemini-1.5-flash-001")
                response = model.generate_content(prompt)
                return f"Aurora Bot: {response.text}"

    return "Aurora Bot: Sorry, I couldn't find that in the town records."

# --- STEP 5: TEST ---
print(ask_aurora_bay("Where is the town hall?"))
print(ask_aurora_bay("Who do I call for wildlife issues?"))

🚀 Starting Challenge 2 Solution (Final Fix)...
--- 1. Loading Data into aurora_bay_rag ---
✅ Loaded 50 rows from GCS.
--- 2. Creating Remote Model (Connection: us-central1.vertex-ai-conn) ---
--- 3. Generating Vectors (Index) ---
✅ Embeddings generated successfully.

User Query: 'Where is the town hall?'
Aurora Bot: Sorry, I couldn't find that in the town records.

User Query: 'Who do I call for wildlife issues?'
Aurora Bot: Sorry, I couldn't find that in the town records.


In [17]:
# --- DEBUGGING TOOL ---

# 1. SEE WHAT IS IN THE DB
print("--- 🔎 INSPECTING KNOWLEDGE BASE (Top 5 Entries) ---")
sql_peek = f"""
SELECT question, answer
FROM `{PROJECT_ID}.{DATASET_ID}.faqs_raw`
LIMIT 5
"""
rows = run_query(sql_peek)
for row in rows:
    print(f"Q: {row.question}")
    print(f"A: {row.answer[:50]}...")
    print("-" * 20)

# 2. RUN A RAW SEARCH (No Threshold)
def debug_search(user_query):
    print(f"\n--- 🧪 DEBUG SEARCH FOR: '{user_query}' ---")

    search_sql = f"""
    SELECT
        base.question,
        base.answer,
        distance, -- The raw distance score (Lower is better)
        (1 - distance) AS similarity -- Calculated similarity
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET_ID}.faq_vectors`,
        'embedding',
        (
            SELECT ml_generate_embedding_result, '{user_query}' AS query
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
                (SELECT '{user_query}' AS content)
            )
        ),
        top_k => 3
    )
    ORDER BY distance ASC; -- Show closest matches first
    """
    results = run_query(search_sql)

    for row in results:
        print(f"Match: {row.question}")
        print(f"Score: {row.similarity:.4f} (Distance: {row.distance:.4f})")
        print(f"Answer: {row.answer[:100]}...\n")

# Run the debug searches
debug_search("town hall")
debug_search("wildlife")

--- 🔎 INSPECTING KNOWLEDGE BASE (Top 5 Entries) ---
Q: When was Aurora Bay founded?
A: Aurora Bay was founded in 1901 by a group of fur t...
--------------------
Q: What is the population of Aurora Bay?
A: Aurora Bay has a population of approximately 3,200...
--------------------
Q: Where is the Aurora Bay Town Hall located?
A: The Town Hall is located at 100 Harbor View Road, ...
--------------------
Q: Who is the current mayor of Aurora Bay?
A: The current mayor is Linda Greenwood, elected in 2...
--------------------
Q: What are the primary industries in Aurora Bay?
A: The primary industries include commercial fishing,...
--------------------

--- 🧪 DEBUG SEARCH FOR: 'town hall' ---
Match: When are the town council meetings held?
Score: 0.1379 (Distance: 0.8621)
Answer: Town council meetings are held every second Tuesday of the month at 6 PM in the Town Hall conference...

Match: Where is the Aurora Bay Town Hall located?
Score: 0.0612 (Distance: 0.9388)
Answer: The Town Hall is loc

In [20]:
# --- STEP 4: RAG CHATBOT (FIXED MODEL NAME) ---
def ask_aurora_bay(user_query):
    print(f"\nUser Query: '{user_query}'")

    # A. Search
    search_sql = f"""
    SELECT
        base.answer,
        (1 - distance) AS similarity_score
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET_ID}.faq_vectors`,
        'embedding',
        (
            SELECT ml_generate_embedding_result, '{user_query}' AS query
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
                (SELECT '{user_query}' AS content)
            )
        ),
        top_k => 1
    )
    ORDER BY similarity_score DESC;
    """
    results = run_query(search_sql)

    # B. Generate
    if results:
        for row in results:
            print(f"   --> Match Score: {row.similarity_score:.4f} (Accepted)")

            prompt = f"""
            You are the Aurora Bay Town Clerk. Answer using this official info ONLY.
            INFO: {row.answer}
            QUESTION: {user_query}
            """

            # FIX: Changed 'gemini-2.5-flash-001' to 'gemini-1.5-flash'
            # This points to the latest stable version automatically
            model = GenerativeModel("gemini-2.5-flash")

            response = model.generate_content(prompt)
            return f"Aurora Bot: {response.text}"

    return "Aurora Bot: Sorry, I couldn't find that in the town records."

# --- STEP 5: TEST ---
print(ask_aurora_bay("Where is the town hall located?"))
print(ask_aurora_bay("Who is the mayor?"))


User Query: 'Where is the town hall located?'
   --> Match Score: 0.2015 (Accepted)
Aurora Bot: I apologize, but the provided information does not specify the location of the Town Hall. It only states that town council meetings are held in the Town Hall conference room.

User Query: 'Who is the mayor?'
   --> Match Score: 0.0248 (Accepted)
Aurora Bot: Linda Greenwood.


In [22]:
# --- STEP 4: RAG CHATBOT (HIGH ACCURACY / TOP-3) ---
def ask_aurora_bay(user_query):
    print(f"\nUser Query: '{user_query}'")

    # A. Search (Get Top 3 matches instead of 1)
    search_sql = f"""
    SELECT
        base.answer,
        (1 - distance) AS similarity_score
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET_ID}.faq_vectors`,
        'embedding',
        (
            SELECT ml_generate_embedding_result, '{user_query}' AS query
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
                (SELECT '{user_query}' AS content)
            )
        ),
        top_k => 3  -- <--- INCREASED TO 3
    )
    ORDER BY similarity_score DESC;
    """
    results = run_query(search_sql)

    # B. Compile Context
    if results:
        # Combine all 3 answers into one big block of text
        context_blob = ""
        for row in results:
            print(f"   --> Found Context (Score: {row.similarity_score:.4f}): {row.answer[:50]}...")
            context_blob += f"- {row.answer}\n"

        # C. Generate (Gemini)
        # We give Gemini all 3 facts and ask it to pick the right one
        prompt = f"""
        You are the Aurora Bay Town Clerk.
        Use the provided KNOWLEDGE BASE to answer the user's question.
        If the answer is not in the Knowledge Base, say "I don't know."

        KNOWLEDGE BASE:
        {context_blob}

        USER QUESTION: {user_query}
        """

        # Sticking to the stable model name
        model = GenerativeModel("gemini-2.5-flash")
        response = model.generate_content(prompt)
        return f"Aurora Bot: {response.text}"

    return "Aurora Bot: Sorry, I couldn't find that in the town records."

# --- STEP 5: TEST AGAIN ---
print(ask_aurora_bay("Where is the town hall located?"))


User Query: 'Where is the town hall located?'
   --> Found Context (Score: 0.2015): Town council meetings are held every second Tuesda...
   --> Found Context (Score: 0.1612): The Town Hall is located at 100 Harbor View Road, ...
   --> Found Context (Score: 0.0568): Building permit applications can be obtained at th...
Aurora Bot: The Town Hall is located at 100 Harbor View Road, in the center of Aurora Bay, close to the main harbor.
